In [1]:
import numpy as np
import pickle
f_handle = open("vocab.pkl", "rb")
vocab = pickle.load(f_handle)
vocab = vocab[2:] # remove <pad>, <unk>

candidate_num = 1000 #候选词数量

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def dummy_linguistic_model(context:list):
    # shuffle
    seed = 0
    for token in context:
        seed += vocab.index(token)
    np.random.seed(seed)
    index = [i for i in range(len(vocab))]
    np.random.shuffle(index)
    candidate_token_index = index[:candidate_num]
    candidate_token_str   = list(map(lambda i: vocab[i], candidate_token_index))
    # make prob
    candidate_probability = softmax(np.random.standard_normal(candidate_num))
    # make dict
    candidate_token_prob_dict = dict(zip(candidate_token_str, candidate_probability))
    return candidate_token_prob_dict

In [4]:
import bitarray
import huffman

In [15]:
def encode_dummy(hidden_msg):
    ba = bitarray.bitarray()
    ba.frombytes(hidden_msg.encode('utf-8'))
    msg_bits_list = ba.tolist()
    msg_bits_length = len(msg_bits_list)

    sentence = []
    i = 0
    encode_msg_bits = []

    while i < msg_bits_length:
        token_prob_dict = dummy_linguistic_model(sentence)

        coding = huffman.HuffmanCoding()
        coding.make_heap(token_prob_dict)
        coding.merge_nodes()

        root = coding.make_codes()

        while root.token is None:
            if i >= msg_bits_length or msg_bits_list[i] == 0:
                root = root.left
            else:
                root = root.right
            i += 1

        selection = root.token
#         print(coding.get_encoded_tokens([selection]))
        sentence += [selection]
        encode_msg_bits += coding.get_encoded_tokens([selection])
    return ' '.join(sentence)

In [72]:
def decode_dummy(sentence):
    decode_msg_bits = ''
    sentence = sentence.split(' ')
    for x in range(len(sentence)):
        token_prob_dict = dummy_linguistic_model(sentence[:x])
        coding = huffman.HuffmanCoding()
        coding.make_heap(token_prob_dict)
        coding.merge_nodes()
        root = coding.make_codes()
        decode_msg_bits += coding.get_encoded_tokens([sentence[x]])
    def decode_from_bits(bits):
        if type(bits) is str:
            bits = list(map(int, bits))
        bits = bits[:int(len(bits)/8) * 8] # utf-8编码中bit长度是8的整数倍
        ba = bitarray.bitarray(bits)
        
        msg_str=ba.tobytes().decode('utf-8','ignore')
        return msg_str
    return decode_from_bits(decode_msg_bits)

In [73]:
sentence = encode_dummy("快进到煎饼果子")

In [74]:
msg = decode_dummy(sentence)

In [75]:
msg

'快进到煎饼果子'